In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import polars as pl
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
df = pl.read_parquet(
    "https://osf.io/gk2ty/download",
    use_pyarrow=True,
)


In [ ]:
for c in df.columns:
    print(c)


In [ ]:
df = df.with_columns(
    pl.col("SLIP_INSERTION_BOOL_MASK").any().over(
        ["Treatment", "Run ID", "Generation Born"],
    ).alias("SLIP_INSERTION_BOOL_MASK any"),
)


In [ ]:
df = df.with_columns(
    pl.col("Is Task Coding Site").any().over(
        ["Treatment", "Run ID", "Generation Born", "Site"],
    )
    .alias("is any coding site"),
)


In [ ]:
df = df.with_columns(
    pl.col("Is Task Coding Site Delta").sum().over(
        ["Treatment", "Run ID", "Generation Born"],
    )
    .alias("is task coding site delta sum"),
)


In [ ]:
df = df.with_columns(
    pl.col("has task").sum().over(
        ["Treatment", "Run ID", "Generation Born", "Site"],
    )
    .alias("num tasks has"),
)


In [ ]:
df = df.with_columns(
    pl.col("is any coding site").sum().over(
        ["Treatment", "Run ID", "Generation Born", "Task",],
    ).alias("num coding sites"),
)


In [ ]:
df = df.with_columns(
    pl.col("Slip Insertion Cumulative Count").sum().over(
        ["Treatment", "Run ID", "Generation Born", "Task",],
    ).alias("net Slip Insertion Cumulative Count"),
)


In [ ]:
dfx = df.group_by(
    ["Treatment", "Run ID", "Generation Born"],
).first()


In [ ]:
dfx = dfx.with_columns(
    (
        pl.col("net Slip Insertion Cumulative Count") / pl.col("Genome Length")
    ).alias("mean Slip Insertion Cumulative Count")
)


In [ ]:
dfx = dfx.with_columns(
    (
        pl.col("num coding sites") / pl.col("Genome Length")
    ).alias("frac coding sites")
)


In [ ]:
tp.tee(
    sns.regplot,
    y="frac coding sites",
    x="mean Slip Insertion Cumulative Count",
    scatter_kws={"alpha": 0.1},
    data=dfx.filter(
        pl.col("Treatment") == "Slip-duplicate",
    ).to_pandas(),
)


In [ ]:
tp.tee(
    sns.regplot,
    y="num coding sites",
    x="mean Slip Insertion Cumulative Count",
    scatter_kws={"alpha": 0.1},
    data=dfx.filter(
        pl.col("Treatment") == "Slip-duplicate",
    ).to_pandas(),
)
